In [323]:
import time
from threads import Threads
from pydantic import BaseModel, ValidationError, create_model

In [12]:
# username=username, password=password
threads = Threads()

In [16]:
threads.public_api.get_thread(id=3141002295235099165)

{'data': {'data': {'containing_thread': {'thread_items': [{'post': {'user': {'profile_pic_url': 'https://scontent.cdninstagram.com/v/t51.2885-19/357376107_1330597350674698_8884059223384672080_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent.cdninstagram.com&_nc_cat=1&_nc_ohc=ljsMOsBUbmkAX-Si4lv&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfCsD-BCrdaAug23dF_ZSJkjerVKx0SB7jCBxirpQg6jUA&oe=64B4C6C0&_nc_sid=10d13b',
        'username': 'zuck',
        'id': None,
        'is_verified': True,
        'pk': '314216'},
       'image_versions2': {'candidates': [{'height': 3000,
          'url': 'https://scontent.cdninstagram.com/v/t51.2885-15/357916630_786901039847062_4398530087245184228_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent.cdninstagram.com&_nc_cat=110&_nc_ohc=Hw22H60YX-cAX8zSFRs&edm=APs17CUBAAAA&ccb=7-5&ig_cache_key=MzE0MTAwMjI5NTIzNTA5OTE2NQ%3D%3D.2-ccb7-5&oh=00_AfDNry8bXwNeIihclLNlnpEgwPgL0pEaA0CMj4X8jEO9lA&oe=64B4A447&_nc_sid=10d13b',
          'width': 4000,
          '__typename': 'XDTImageCandidate'},
  

In [338]:
user_id = 4

In [339]:
user_data = threads.public_api.get_user(id=user_id)
user_data

{'data': {'userData': {'user': {'is_private': False,
    'profile_pic_url': 'https://scontent.cdninstagram.com/v/t51.2885-19/357806502_947628763167341_5399985934905782773_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent.cdninstagram.com&_nc_cat=105&_nc_ohc=Mei7UUCY8MQAX9_dzHy&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfAW9qJ4Dyyi7fLxwgCrDZif2mms7T5PHLtmmQ5FAdDE_A&oe=64B3FE00&_nc_sid=10d13b',
    'username': 'mikeyk',
    'hd_profile_pic_versions': [{'height': 320,
      'url': 'https://scontent.cdninstagram.com/v/t51.2885-19/357806502_947628763167341_5399985934905782773_n.jpg?stp=dst-jpg_s320x320&_nc_ht=scontent.cdninstagram.com&_nc_cat=105&_nc_ohc=Mei7UUCY8MQAX9_dzHy&edm=APs17CUBAAAA&ccb=7-5&oh=00_AfDg8X_mxtyhcX1p7Eh_gXRJad_A-OSZVXq1lSVbRZY3kg&oe=64B3FE00&_nc_sid=10d13b',
      'width': 320},
     {'height': 640,
      'url': 'https://scontent.cdninstagram.com/v/t51.2885-19/357806502_947628763167341_5399985934905782773_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent.cdninstagram.com&_nc_cat=105&_nc_ohc=Mei

In [340]:
def process_user_data(user_data: dict) -> list:
    """Extracts important fields from the user response"""
    
    if 'data' in user_data.keys():
        user_data = user_data['data']
        
    if 'userData' in user_data.keys():
        user_data = user_data['userData']
    
    if 'user' in user_data.keys():
        user_data = user_data['user']
    
    biography_entities = []
    if 'biography_with_entities' in user_data:
        if 'entities' in biography_entities:
            biography_entities = biography_entities['entities']
        else:
            biography_entities = []
    
    user_id = user_data['pk'] if user_data['id'] is None else user_data['id'] 
    
    keys_to_drop = ['profile_pic_url', 'hd_profile_pic_versions', 'biography_with_entities', 'profile_context_facepile_users', 'id']
    for key in keys_to_drop:
        try:
            user_data.pop(key)
        except KeyError:
            pass

    user_data['user_id'] = user_id
    user_data['biography_entities'] = biography_entities
    
    return user_data

In [341]:
user = process_user_data(user_data)
user

{'is_private': False,
 'username': 'mikeyk',
 'is_verified': True,
 'biography': 'Co-founder of Instagram and former CTO. Working on Artifact.',
 'follower_count': 56801,
 'bio_links': [{'url': 'https://artifact.news'}],
 'pk': '4',
 'full_name': 'Mike Krieger',
 'user_id': '4',
 'biography_entities': []}

In [342]:
user_threads = threads.public_api.get_user_threads(id=user['pk'])

In [343]:
def process_user_threads(thread_data: dict):
    """Extracts important fields from each thread response"""
    if 'data' in thread_data.keys():
        thread_data = thread_data['data']
    
    if 'mediaData' in thread_data.keys():
        thread_data = thread_data['mediaData']
        
    if 'threads' in thread_data.keys():
        thread_data = thread_data['threads']
        
    processed_threads = []
    for item in thread_data:
        for thread_item in item['thread_items']:
            post = thread_item['post']
            thread_id = post['id']
            
            username = post['user']['username']
            user_id = post['user']['pk']
            user_verified = post['user']['is_verified']
            
            image_height = post['original_height']
            image_width = post['original_width']
            has_audio = False if post['has_audio'] is None else True
            
            quoted = post['text_post_app_info']['share_info']['quoted_post']
            reposted = post['text_post_app_info']['share_info']['reposted_post']
            is_post_unavailable = post['text_post_app_info']['is_post_unavailable']
            was_post_taken_down = not is_post_unavailable
            
            text = post['caption']['text']
            
            # Debugging
            for k in post['caption'].keys():
                if k != 'text':
                    print(f'Found something in the post caption that was not text: {k} | thread_id: {thread_id}')
            
            posted_timestamp = post['taken_at']
            posted_datetime = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(posted_timestamp))
            
            likes = post['like_count']
        
            processed_threads.append({
                'thread_id': thread_id,
                'username': username,
                'user_id': user_id,
                'is_user_verified': user_verified,
                'image_height': image_height,
                'image_width': image_width,
                'has_audio': has_audio,
                'quoted_count': quoted,
                'reposted_count': reposted,
                'likes_count': likes,
                'is_post_unavailable': is_post_unavailable,
                'was_post_taken_down': was_post_taken_down,
                'text': text,
                'posted_timestamp': posted_timestamp,
                'posted_datetime': posted_datetime
            })
    
    return processed_threads

In [344]:
data = process_user_threads(user_threads)

In [345]:
data

[{'thread_id': '3144620576332564767_4',
  'username': 'mikeyk',
  'user_id': '4',
  'is_user_verified': True,
  'image_height': 612,
  'image_width': 612,
  'has_audio': False,
  'quoted_count': None,
  'reposted_count': None,
  'likes_count': 71,
  'is_post_unavailable': False,
  'was_post_taken_down': True,
  'text': 'Anyone else heading to Vegas for the F1 GP this November? Excited to see them head down the Strip, should be quite the show.',
  'posted_timestamp': 1689088006,
  'posted_datetime': '2023-07-11 15:06:46'},
 {'thread_id': '3141175247941155918_4',
  'username': 'mikeyk',
  'user_id': '4',
  'is_user_verified': True,
  'image_height': 612,
  'image_width': 612,
  'has_audio': False,
  'quoted_count': None,
  'reposted_count': None,
  'likes_count': 255,
  'is_post_unavailable': False,
  'was_post_taken_down': True,
  'text': 'What are your favorite books so far this year? Here are my four fav novels I’ve read.',
  'posted_timestamp': 1688677291,
  'posted_datetime': '2023-

In [346]:
user

{'is_private': False,
 'username': 'mikeyk',
 'is_verified': True,
 'biography': 'Co-founder of Instagram and former CTO. Working on Artifact.',
 'follower_count': 56801,
 'bio_links': [{'url': 'https://artifact.news'}],
 'pk': '4',
 'full_name': 'Mike Krieger',
 'user_id': '4',
 'biography_entities': []}

In [347]:
class UserModel(BaseModel):
    pk: str
    user_id: str | None
    username: str
    
    full_name: str | None
    biography: str | None
    is_verified: bool
    follower_count: int
    biography_entities: list | None
    bio_links: list[dict[str, str]]
    is_private: bool
    
    class Config:
        populate_by_name = True
        json_schema_extra = {
            "example": {
                'is_private': False,
                'username': 'mikeyk',
                'is_verified': True,
                'biography': 'Co-founder of Instagram and former CTO. Working on Artifact.',
                'follower_count': 56792,
                'bio_links': [{'url': 'https://artifact.news'}],
                'pk': '4',
                'full_name': 'Mike Krieger',
                'id': None,
                'biography_entities': []
            }
        }

In [348]:
user = UserModel(**user)

In [349]:
user

UserModel(pk='4', user_id='4', username='mikeyk', full_name='Mike Krieger', biography='Co-founder of Instagram and former CTO. Working on Artifact.', is_verified=True, follower_count=56801, biography_entities=[], bio_links=[{'url': 'https://artifact.news'}], is_private=False)

In [350]:
data[0]

{'thread_id': '3144620576332564767_4',
 'username': 'mikeyk',
 'user_id': '4',
 'is_user_verified': True,
 'image_height': 612,
 'image_width': 612,
 'has_audio': False,
 'quoted_count': None,
 'reposted_count': None,
 'likes_count': 71,
 'is_post_unavailable': False,
 'was_post_taken_down': True,
 'text': 'Anyone else heading to Vegas for the F1 GP this November? Excited to see them head down the Strip, should be quite the show.',
 'posted_timestamp': 1689088006,
 'posted_datetime': '2023-07-11 15:06:46'}

In [351]:
data[1]

{'thread_id': '3141175247941155918_4',
 'username': 'mikeyk',
 'user_id': '4',
 'is_user_verified': True,
 'image_height': 612,
 'image_width': 612,
 'has_audio': False,
 'quoted_count': None,
 'reposted_count': None,
 'likes_count': 255,
 'is_post_unavailable': False,
 'was_post_taken_down': True,
 'text': 'What are your favorite books so far this year? Here are my four fav novels I’ve read.',
 'posted_timestamp': 1688677291,
 'posted_datetime': '2023-07-06 21:01:31'}

In [352]:
data[2]

{'thread_id': '3140545763160075590_4',
 'username': 'mikeyk',
 'user_id': '4',
 'is_user_verified': True,
 'image_height': 794,
 'image_width': 640,
 'has_audio': True,
 'quoted_count': None,
 'reposted_count': None,
 'likes_count': 202,
 'is_post_unavailable': False,
 'was_post_taken_down': True,
 'text': "Fun update last week in artifact_news: our reading level cards just got an interactive 3D makeover. Visit your profile, tap your current level, and see the magic for yourself. Curious how we did it on iOS and Android? Here's the thread: 🧵",
 'posted_timestamp': 1688602263,
 'posted_datetime': '2023-07-06 00:11:03'}

In [353]:
data[3]

{'thread_id': '3140546144073699340_4',
 'username': 'mikeyk',
 'user_id': '4',
 'is_user_verified': True,
 'image_height': 1440,
 'image_width': 1920,
 'has_audio': False,
 'quoted_count': None,
 'reposted_count': None,
 'likes_count': 48,
 'is_post_unavailable': False,
 'was_post_taken_down': True,
 'text': 'To start, our design wizard gunnargray built the original cards in Cinema4D. We wanted the cards to look distinct from each other as you level up, and for the "higher level" cards to feel premium/metallic.',
 'posted_timestamp': 1688602296,
 'posted_datetime': '2023-07-06 00:11:36'}

In [355]:
class Thread(BaseModel):
    thread_id: str
    username: str
    user_id: str
    is_user_verified: bool
    image_height: int | None
    image_width: int | None
    has_audio: bool
    quoted_count: int | None
    reposted_count: int | None
    likes_count: int | None
    is_post_unavailable: bool | None
    was_post_taken_down: bool | None
    text: str | None
    posted_timestamp: int | None
    posted_datetime: str | None
        
    class Config:
        populate_by_name = True
        json_schema_extra = {
            "example": {
                'thread_id': '3140546144073699340_4',
                'username': 'mikeyk',
                'user_id': '4',
                'is_user_verified': True,
                'image_height': 1440,
                'image_width': 1920,
                'has_audio': False,
                'quoted_count': None,
                'reposted_count': None,
                'likes_count': 48,
                'is_post_unavailable': False,
                'was_post_taken_down': True,
                'text': 'To start, our design wizard gunnargray built the original cards in Cinema4D. We wanted the cards to look distinct from each other as you level up, and for the "higher level" cards to feel premium/metallic.',
                'posted_timestamp': 1688602296,
                'posted_datetime': '2023-07-06 00:11:36'
            }
        }

In [357]:
thread = Thread(**data[0])

In [358]:
thread

Thread(thread_id='3144620576332564767_4', username='mikeyk', user_id='4', is_user_verified=True, image_height=612, image_width=612, has_audio=False, quoted_count=None, reposted_count=None, likes_count=71, is_post_unavailable=False, was_post_taken_down=True, text='Anyone else heading to Vegas for the F1 GP this November? Excited to see them head down the Strip, should be quite the show.', posted_timestamp=1689088006, posted_datetime='2023-07-11 15:06:46')